In [1]:
import pandas as pd
import numpy as np
import keras
from keras import callbacks
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input,LSTM, Flatten, Dense, Activation, Dropout, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
#import keras_metrics
#import nltk
#from nltk.tokenize import word_tokenize
#import pickle
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
np.random.seed(42)
#import imblearn
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [2]:
df_train = pd.read_csv("output_files/combined.csv", index_col=0)

In [3]:
df_train.head()

,ART_CODE,DESC_CODED,HLGT_NAME_COMPL,HLT_NAME_COMPL,INC_CODE,INC_CODE_J,LLT_NAME_COMPL,PT_NAME_COMPL,REPORTED_TERM,SOC_CODE,len
0,0,Hyponatraemia,ELECTROLYTE AND FLUID BALANCE CONDITIONS,SODIUM IMBALANCE,10021038.0,10021036,HYPONATREMIA,HYPONATRAEMIA,HYPONATREMIA,10027433.0,1
1,1,Subacute cutaneous lupus erythematosus,EPIDERMAL AND DERMAL CONDITIONS,CONNECTIVE TISSUE DISORDERS,10057903.0,10057903,SUBACUTE CUTANEOUS LUPUS ERYTHEMATOSUS,SUBACUTE CUTANEOUS LUPUS ERYTHEMATOSUS,OMEPRAZOLE INDUCED SUBACUTE CUTANEOUS LUPUS ER...,10040785.0,4
2,2,Blood bilirubin unconjugated increased,HEPATOBILIARY INVESTIGATIONS,LIVER FUNCTION ANALYSES,10021709.0,10021709,INDIRECT BILIRUBIN INCREASED,BLOOD BILIRUBIN UNCONJUGATED INCREASED,INDIRECT BILIRUBIN (74.7 MICROMOL/L),10022891.0,4
3,3,toxic epidermal necrolysis,EPIDERMAL AND DERMAL CONDITIONS,BULLOUS CONDITIONS,10044223.0,10044223,TOXIC EPIDERMAL NECROLYSIS,TOXIC EPIDERMAL NECROLYSIS,TOXIC EPIDERMAL NECROLYSIS,10040785.0,3
4,4,Bradycardia,CARDIAC ARRHYTHMIAS,RATE AND RHYTHM DISORDERS NEC,10006093.0,10006093,BRADYCARDIA,BRADYCARDIA,BRADYCARDIA,10007541.0,1


In [4]:
X, y = df_train.REPORTED_TERM, df_train.ART_CODE

print(X.shape)
print(y.shape)

(63233,)
(63233,)


In [5]:
MAX_WORDS = 63000
MAX_LEN = 200

In [6]:
def preprocess(X):
    tok = Tokenizer(num_words=MAX_WORDS)
    tok.fit_on_texts(X)
    sequences = tok.texts_to_sequences(X)
    sequences_matrix = sequence.pad_sequences(sequences, maxlen=MAX_LEN)
    return sequences_matrix

In [7]:
X = preprocess(X)

In [8]:
dummies = pd.get_dummies(y.values)

In [9]:
y = np.array(dummies)

In [10]:
no_categories = y.shape[1]

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, shuffle=True, 
                                                  random_state=42)

In [12]:
def RNN():
    inputs = Input(name='inputs',shape=[MAX_LEN])
    layer = Embedding(MAX_WORDS,100,input_length=MAX_LEN)(inputs)
    layer = LSTM(128)(layer)
    layer = Dense(512,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.30)(layer)
    layer = Dense(no_categories,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [13]:
model = RNN()
#model.summary()
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, batch_size=128,epochs=15, validation_data=[X_val, y_val], shuffle=True)

Train on 50586 samples, validate on 12647 samples
Epoch 1/15
50586/50586 [==============================] - 316s 6ms/step - loss: 6.6246 - acc: 0.0483 - val_loss: 6.0587 - val_acc: 0.0681
Epoch 2/15
50586/50586 [==============================] - 315s 6ms/step - loss: 5.5775 - acc: 0.1407 - val_loss: 5.1137 - val_acc: 0.2144
Epoch 3/15
50586/50586 [==============================] - 313s 6ms/step - loss: 4.6642 - acc: 0.2815 - val_loss: 4.5064 - val_acc: 0.3367
Epoch 4/15
50586/50586 [==============================] - 312s 6ms/step - loss: 4.1192 - acc: 0.3744 - val_loss: 4.1643 - val_acc: 0.3931
Epoch 5/15
50586/50586 [==============================] - 312s 6ms/step - loss: 3.7729 - acc: 0.4307 - val_loss: 3.9507 - val_acc: 0.4447
Epoch 6/15
50586/50586 [==============================] - 312s 6ms/step - loss: 3.5259 - acc: 0.4693 - val_loss: 3.8294 - val_acc: 0.4625
Epoch 7/15
50586/50586 [==============================] - 314s 6ms/step - loss: 3.3110 - acc: 0.5048 - val_loss: 3.5854 - 

In [16]:
model.save('output_files/lstm_v1.h5')